# SQL Programming - Subqueries using Select Statements

## 1.0 Connecting to our Database

In [ ]:
# We will first load the sql extension into our environment
%load_ext sql

# Then connect to our in memory sqlite database
# NB: This database will cease to exist as soon as the database connection is closed. 
%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @None'

## 1.1 Loading Data from CSV Files

Datasets used in this notebook:


1.   **World Countries**

> ([Dataset Download Link](https://drive.google.com/a/moringaschool.com/file/d/1XlabneVWhvbmpRUi_Wix2FR5uYAAp-ZY/view?usp=sharing))

2.   **World Cities** 

> ([Dataset Download Link](https://drive.google.com/a/moringaschool.com/file/d/1R5952m-8hH91uiwDK8dcL9sT2D_PEcuw/view?usp=sharing))






In [ ]:
# Importing Pandas
import pandas as pd

In [ ]:
# Loading our Countries dataset
# 
with open('world_countries.csv','r') as f:
    world_countries = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql DROP TABLE if EXISTS world_countries;
%sql PERSIST world_countries;
%sql SELECT * FROM world_countries ;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


country,population,EU,coastline
Albania,2.9,no,yes
Andorra,0.07,no,no
Austria,8.57,yes,no
Belarus,9.48,no,no
Belgium,11.37,yes,yes


In [ ]:
# Loading our cities dataset 
# 
with open('world_cities.csv','r') as f:
    world_cities = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql DROP TABLE if EXISTS world_cities;
%sql PERSIST world_cities;
%sql SELECT * FROM world_cities;

## 1.2 Subqueries using the SELECT statement

A subquery is a query within another query. The outer query is called as main query and inner query is called as subquery. The subquery must be enclosed with parenthesis and generally executes first, and its output is used to complete the query condition for the main or outer query. We will go through examples of how subqueries can be used to answers different types of questions then later, we will use our learnings to work on the challenges.

In [ ]:
# Example 1
# Subqueries can occur in the select list of the containing query
# as shown below;
# Let's find all countries in the Countries table with no city in the Cities table
# 
%%sql
SELECT country
FROM world_Countries
WHERE NOT EXISTS (SELECT * FROM world_Cities
                  WHERE world_Cities.country = world_Countries.country) 


 * sqlite://
Done.


country
Cyprus
Iceland
Kosovo
Liechtenstein
Luxembourg


In [ ]:
# Example 2
# Find the European countries with cities experiencing temperature > 20?
# 
%%sql
SELECT country
FROM world_Countries
WHERE EU = 'yes'
AND EXISTS (SELECT * FROM world_Cities
            WHERE world_Cities.country = world_Countries.country
            AND temperature > 15)

 * sqlite://
Done.


country
Greece
Italy
Portugal
Spain


In [ ]:
# Example 3: Method 1
# Find the westernmost city, returning the city and longitude
# 
%%sql
SELECT city, longitude
FROM world_Cities C1
WHERE NOT EXISTS (SELECT * FROM world_Cities C2
                  WHERE C2.longitude < C1.longitude)

 * sqlite://
Done.


city,longitude
Lisbon,-9.14


In [ ]:
# Example 3: Method 2
# Find the westernmost city, returning the city and longitude
# 
%%sql
SELECT city, longitude
FROM world_Cities
WHERE longitude = (SELECT min(longitude) FROM world_Cities)

 * sqlite://
Done.


city,longitude
Lisbon,-9.14


In [ ]:
# Example 4:
# How many cities are in the EU?
# 
%%sql
SELECT count()
FROM world_Cities
WHERE country IN 
               (SELECT country FROM world_Countries WHERE EU = 'yes')

 * sqlite://
Done.


count()
150


### <font color="green">1.2 Challenges</font>

In [ ]:
# Challenge 1
# Find number of countries that have a city with latitude > 1000 (start with country list)
# 
%%sql
SELECT count(country)
FROM world_countries
WHERE country IN(SELECT city FROM world_cities WHERE latitude > 1000)

 * sqlite://
Done.


count(country)
0


In [ ]:
# Challenge 2
# Find three western most cities returning both the cities and their longitude
#
%%sql
SELECT city, longitude
FROM world_Cities
WHERE longitude = (SELECT min(longitude) FROM world_Cities) LIMIT 3;

 * sqlite://
Done.


city,longitude
Lisbon,-9.14


In [ ]:
# Challenge 3
# Find cities with temperature is more than 50% higher than the average; 
# We will return the city, country, and temperature, ordered by descending temperature
# Hint: avg(temperature) * 1.5
# 
%%sql
SELECT city, longitude,temperature
FROM world_Cities
WHERE temperature < (SELECT (temperature*1.5) FROM world_Cities)ORDER BY temperature DESC;

In [ ]:
# Challenge 4
# How many cities are not in the EU and without a coastline?
# Hint: "not in"
# 
%%sql
SELECT COUNT(city) FROM world_Cities
WHERE city =
(SELECT country
FROM world_Countries
WHERE EU = 'no' AND coastline="no")


 * sqlite://
Done.


COUNT(city)
1


In [ ]:
# Challenge 5
# Find the number of countries with no coastline and a city with longitude < 10
# 
%%sql
SELECT count()
FROM world_Countries
WHERE country IN 
               (SELECT city FROM world_cities WHERE coastline="no" AND longitude<10)

 * sqlite://
Done.


count()
1


In [ ]:
# Challenge 6
# Find countries in Countries table with no city in Cities table
# 
%%sql
SELECT count()
FROM world_Countries
WHERE country NOT EXISTS 
               (SELECT city FROM world_cities )



 * sqlite://
(sqlite3.OperationalError) near "EXISTS": syntax error
[SQL: SELECT count()
FROM world_Countries
WHERE country NOT EXISTS 
               (SELECT city FROM world_cities )]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [ ]:
# Challenge 7
# Find all cities in a country whose population is < 2, returning the city and country.
# 
%%sql
SELECT city 
FROM world_Cities INNER JOIN world_Countries ON world_Cities.country = world_Countries.country
WHERE population <2;

 * sqlite://
Done.


city
Andorra
Daugavpils
Podgorica
Riga
Tallinn
Tartu


In [ ]:
# Challenge 8
# Find all countries with no city having a temperature > 6
# 
%%sql
SELECT  
FROM world_Cities INNER JOIN world_Countries ON world_Cities.country = world_Countries.country
WHERE temperature >6 AND city NOT EXISTS;

 * sqlite://
(sqlite3.OperationalError) near "EXISTS": syntax error
[SQL: SELECT * 
FROM world_Cities INNER JOIN world_Countries ON world_Cities.country = world_Countries.country
WHERE temperature >6 AND city NOT EXISTS;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)
